In [1]:
import kfp
from kfp import components
import os

In [3]:
DATA_QUALITY = os.path.abspath("")
EVIDENTLY    = os.path.dirname(DATA_QUALITY)
HTML_VIWER   = os.path.join(EVIDENTLY, 'html-viewer')

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"
# KFP_CLIENT = kfp.Client()


# with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace") as f:
#     NAMESPACE = f.read()
# NAMESPACE

In [ ]:
chicago_taxi_dataset_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/e3337b8bdcd63636934954e592d4b32c95b49129/components/datasets/Chicago%20Taxi/component.yaml')
data_quality_op         = components.load_component_from_file(os.path.join(DATA_QUALITY, 'component.yaml'))
html_viewer_op          = components.load_component_from_file(os.path.join(HTML_VIWER, 'component.yaml'))


In [13]:
@kfp.dsl.pipeline(name='test-pipeline')
def test_pipeline():
    training_data_csv = chicago_taxi_dataset_op(
        where='trip_start_timestamp >= "2019-01-01" AND trip_start_timestamp < "2019-02-01"',
        select='tips,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total',
        limit=10000,
    ).output
    
    report = data_quality_op(
        cur=training_data_csv
    ).output
    
    html_viewer_op(
        html=report
    )


In [ ]:
kfp.Client().create_run_from_pipeline_func(
    test_pipeline, arguments={}
)